<a href="https://colab.research.google.com/github/OnurcanKoken/Google-Colaboratory-General-Usage/blob/master/Huber_Loss_Custom_Loss_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huber Loss Function

Coursera - custom-models-layers-loss-functions-with-tensorflow - Week 2

Onurcan Köken - 10th of May, 2021

**How to create custom loss functions?**


Huber Loss: https://en.wikipedia.org/wiki/Huber_loss


Reference: https://www.coursera.org/learn/custom-models-layers-loss-functions-with-tensorflow/ungradedLab/NvHNF/huber-loss-lab/lab

# Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
from tensorflow import keras

## Prepare the Data

Our dummy dataset is just a pair of arrays `xs` and `ys` defined by the relationship $y = 2x - 1$. `xs` are the inputs while `ys` are the labels.

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Training the model

Let's build a simple model and train using a built-in loss function like the `mean_squared_error`.

In [3]:
# using single Dense with a single unit in it
# linear relationship, so you only need 1 unit
# then you learn the weight and the bias
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

print(model.predict([10.0]))
# result will be (10*2) - 1 = 19

[[18.987066]]


### Custom Loss

Now let's see how we can use a custom loss. We first define a function that accepts the ground truth labels (`y_true`) and model predictions (`y_pred`) as parameters. We then compute and return the loss value in the function definition.

In [4]:
def my_huber_loss(y_true, y_pred):
    threshold = 1 # delta symbol in the equation
    error = y_true - y_pred # corresponds to "a"
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss) # boolean to check, value if True, value if False

Using the loss function is as simple as specifying the loss function in the `loss` argument of `model.compile()`.

In [5]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss)
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

[[18.585327]]
